In [25]:
import pandas as pd
from kedro.pipeline.modular_pipeline import pipeline
from metro_sp_mdp.pipelines import mdp

import metro_sp_mdp.pipelines.mdp as sp_mdp

In [11]:
df = pd.read_csv("/home/dusodeth/metrosp_stations.csv")

In [12]:
df

,Unnamed: 0,name,station,lat,lon,line,neigh
0,aacd-servidor,Aacd Servidor,aacd-servidor,-23.597825,-46.652374,['lilas'],"['moema', 'hospital-sao-paulo']"
1,adolfo-pinheiro,Adolfo Pinheiro,adolfo-pinheiro,-23.650073,-46.704206,['lilas'],"['largo-treze', 'alto-da-boa-vista']"
2,alto-da-boa-vista,Alto Da Boa Vista,alto-da-boa-vista,-23.641625,-46.699434,['lilas'],"['adolfo-pinheiro', 'borba-gato']"
3,alto-do-ipiranga,Alto Do Ipiranga,alto-do-ipiranga,-23.602237,-46.612486,['verde'],"['santos-imigrantes', 'sacoma']"
4,ana-rosa,Ana Rosa,ana-rosa,-23.581871,-46.638104,"['azul', 'verde']","['paraiso', 'vila-mariana', 'paraiso', 'chacar..."
...,...,...,...,...,...,...,...
74,vila-mariana,Vila Mariana,vila-mariana,-23.589178,-46.634668,['azul'],"['ana-rosa', 'santa-cruz']"
75,vila-matilde,Vila Matilde,vila-matilde,-23.531917,-46.530873,['vermelha'],"['guilhermina-esperanca', 'penha']"
76,vila-prudente,Vila Prudente,vila-prudente,-23.584595,-46.583437,"['prata', 'verde']","['oratorio', 'tamanduatei']"
77,vila-tolstoi,Vila Tolstoi,vila-tolstoi,-23.600902,-46.527121,['prata'],"['camilo-haddad', 'vila-uniao']"


In [19]:
pipe = pipeline(mdp.create_pipeline(),
                parameters={
                    "params:metro_sp_from":"luz", 
                    "params:metro_sp_to":"carrão"
                })

In [23]:
pipe

Pipeline([
Node(tratamento, 'estacoes', 'estacoes_limpa', 'node_tratamento'),
Node(carrega_mdp, ['estacoes_limpa', 'luz', 'carrão'], 'MDP', 'node_carrega_mdp'),
Node(resolve_mdp, 'MDP', 'json_resultado', 'node_resolve_mdp')
])

<br/>

In [31]:
metro_from = "Estação da Luz".lower() #request.form['metro_from']
metro_to = "Estação Vila Prudente".lower() #request.form['metro_to']

ruido = ["estação", "estacao", "station", "da", "de", "do"]
for r in ruido:
    if r in metro_from:
        metro_from = metro_from.replace(r, "")
    if r in metro_to:
        metro_to = metro_to.replace(r, "")

df = pd.read_csv("~/metrosp_stations.csv")

metro_from_fuzz = sp_mdp.nodes.fuzz_string(str(metro_from), df['station'])
metro_to_fuzz = sp_mdp.nodes.fuzz_string(str(metro_to), df['station'])

In [32]:
print(metro_from_fuzz)
print(metro_to_fuzz)

luz
vila-prudente


In [33]:
sistema = sp_mdp.nodes.Problem(df, metro_from_fuzz, metro_to_fuzz)

In [35]:
sp_mdp.pipeline.resolve_mdp(sistema)

{'estacoes': ['luz',
  'sao-bento',
  'se',
  'japao-liberdade',
  'sao-joaquim',
  'vergueiro',
  'paraiso',
  'ana-rosa',
  'chacara-klabin',
  'santos-imigrantes',
  'alto-do-ipiranga',
  'sacoma',
  'tamanduatei',
  'vila-prudente']}

.